<center>
<img src="https://habrastorage.org/webt/jq/gc/r5/jqgcr5azmatev6zvgufn8xftehy.png">
    
##  II Escuela de Verano en Física Computacional

Autor: [Gerardo Rivera](https://github.com/DangoMelon).\
Institución: Instituto Geofísico del Perú / UNMSM\
Correo: gerardo.rivera1@unmsm.edu.pe
    
Este material está sujeto a los términos y condiciones de la licencia [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Se permite el uso gratuito para cualquier propósito no comercial.

*También puede consultar la última versión de este notebook en nuestro [repositorio](https://github.com/PCPUNMSM) y los videos de clase [en nuestro canal de Youtube](https://www.youtube.com/channel/UCze8vzQLcplutz0nWDNjFCA).*
    

# <center> Sesión 13. Manejo de datos n-dimensionales en python: *conociendo xarray*  

<br>
<div align="center">
    <img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" width=30%>
</div>
<br>

## Outline
1. [Conociendo xarray](#1.-Conociendo-xarray)
2. [Funciones Básicas](#2.-Funciones-básicas)
3. [Trabajando con datos](#3.-Trabajando-con-datos)
4. [Gráficos](#4.-Gráficos)
5. [Integración con dask](#5.-Integración-con-dask)
6. [Aplicación con datos reales](#6.-Aplicación-con-datos-reales)
7. [Aplicación Numérica](#7.-Aplicación-Numérica)

## 1. Conociendo xarray

Para trabajar con data cubes usando python haremos uso extensivo de los paquetes estrella de la plataforma Pangeo. Pangeo es un esfuerzo de la comunidad que promueve la reproductibilidad, escalamiento y transparencia en la ciencia.

El paquete de cabecera para el manejo de dichas estructuras de datos será xarray. Xarray extiende las capacidades de numpy en el manejo de datos n-dimensionales introduciendo etiquetas en forma de dimensiones, coordenadas y atributos que facilitan las operaciones sobre los mismos. El motor para las operaciones usando las etiquetas es proporcionado por pandas; la capacidad de manejar datos n-dimensionales con facilidad es propocionado por numpy; y la escalabilidad de los datos es proporcionado por dask, convirtiendo a este paquete en una herramienta poderosa.

In [ ]:
# Quitar el comentario y correr esta línea si se esta usando google colab
#!pip install -q -U git+https://github.com/pydata/xarray.git#egg=xarray[complete]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

### 1.1. Estructuras de xarray

`xarray` cuenta con dos estructuras simples que son fundamentales comprender para un mejor uso del paquete. Esta es una brevisima descripción sobre algunos conceptos claves que se deben tener presentes, mayor informacion la pueden encontrar siempre en la [documentacion](http://xarray.pydata.org/en/stable/data-structures.html)

#### 1.1.1. `DataArray`

Un `DataArray` es virtualmente igual a un arreglo n-dimensional de numpy que cuenta con etiquetas en sus dimensiones, siendo este la estructura de datos base de xarray. Para denifinir un DataArray necesitaremos proporcionar, como mínimo, una arreglo de elementos, como los generados por `numpy`.

In [ ]:
xr.DataArray(np.random.randn(10, 5))

El `repr` en html de xarray es bien practico cuando estamos trabajando en notebooks, sin embargo, el `repr` en puro texto contiene un poco de información adicional sobre el estado de las coordenadas en el `DataArray` recién creado

In [ ]:
with xr.set_options(display_style="text"):
    display(xr.DataArray(np.random.randn(10, 5)))

De manera automática, nuestras dimensiones fueron asignadas con nombres genéricos ya que no agregamos ningun detalle adicional. Adicionalmente, en el `repr` de texto podemos observar un mensaje adicional sobre `Dimensions without coordinates`, esto se debe a que no tenemos ninguna coordenada definida para nuestras dimensiones. Las coordenadas pueden ser entendidas como etiquetas a cada elemento posicional en una dimensión.  

Ahora crearemos un `DataArray` con la información que hace falta

In [ ]:
# Creamos las coordenadas para nuestras dimensiones
lat = np.arange(-90, 90, 0.25)
lon = np.arange(0, 360, 0.25)

# Creamos un poco de data falsa
llon, llat = np.meshgrid(lon, lat)
r = np.sqrt(llon ** 2 + llat ** 2)
data = np.sin(r)

# Creamos el DataArray
xarr = xr.DataArray(data, coords=[lat, lon], dims=["lat", "lon"], name="false_data")
xarr

Nuestro `DataArray` ha sido creado dentro de la variable xarr con las dimensiones y coordenadas asignadas correctamente. Ahora podemos acceder a los métodos que xarray ofrece para sus objetos.

In [ ]:
xarr.plot()

Existen varias formas convenientes de crear un `DataArray` que son practicas en algunos casos

In [ ]:
xr.zeros_like(xarr)

**Ejercicio**

Crear un `DataArray` llamado "temperatura" a partir de datos aleatorios.

- Debe tener tres coordenadas con nombres "latitud", "longitud" y "profundidad"
- Debe tener atributos en cada coordenada asi como atributos generales

In [ ]:
xr.DataArray()

#### 1.1.2. `Dataset`

Un `Dataset` es la representación en la memoria del sistema de un archivo netcdf. Al igual que en pandas un `DataFrame` es un conjunto de `Series`, en xarray un `Dataset` es un conjunto de `DataArray`'s. Su declaración es un poco más extensa que un `DataArray` pero a cambio proporciona un contenedor a varias variables.

<div align="center">
<img src="http://xarray.pydata.org/en/stable/_images/dataset-diagram.png" width=80%>
<br>
<a href="http://xarray.pydata.org/en/stable/data-structures.html#dataset">Documentación de Dataset</a>
</div>

In [ ]:
# Definimos las coordenadas para nuestras dimensiones
# en este caso tendremos 3 dimensiones: latitud, longitud y tiempo
lat = np.arange(-15, 15, 0.25)
lon = np.arange(140, 280, 0.25)
time = pd.date_range("2019-01-01", freq="D", periods=120)

# Datos aleatorios
temp = np.random.rand(120, len(lat), len(lon))
precip = np.random.rand(120, len(lat), len(lon))
elev = np.random.rand(len(lat), len(lon))
serie = np.random.rand(len(time))

In [ ]:
# Declaramos el Dataset
ds = xr.Dataset(
    {
        "temp": (["time", "lat", "lon"], temp),
        "precip": (["time", "lat", "lon"], precip),
        "elev": (["lat", "lon"], elev),
        "serie": (["time"], serie),
    },
    coords={"time": time, "lat": lat, "lon": lon},
)
ds

Si la terminología es un poco dificil de entender, siempre pueden consultar la [documentación](http://xarray.pydata.org/en/stable/terminology.html0)

### 1.2. Seleccion de datos

`xarray` cuenta con dos métodos para selección de datos: `sel` y `isel`

`sel` funciona usando los nombres (o etiquetas) asignados a cada dimension

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
air = ds.air

In [ ]:
air.sel(time="2013-04-01")

usando `slice` podemos seleccionar un rango de valores

In [ ]:
air.sel(time=slice("2013-04-01", "2013-09-15"))

Tampoco es necesario ser precisos con el valor de las coordenadas

In [ ]:
air.sel(lat=70.5, lon=211, method="nearest")

`isel` funciona usando los indices posicionales, pero aun usando los nombres de las dimensiones

In [ ]:
air.isel(time=10)

In [ ]:
air.isel(lat=slice(10, 30))

In [ ]:
air.isel(lat=10, lon=20)

Esta forma de seleccionar los datos, usando el nombre de la dimensión, es de lejos mucho mejor que hacer indexing de la manera clásica

```python
air[10:40,10:40,14]
```

A simple vista uno no puede saber que valores esta seleccionando

## 2. Funciones básicas

Muchas de las funciones disponibles en `xarray` son tomadas de `numpy` y `pandas`, presentando el mismo comportamiento en muchos casos.  

El `Dataset` que hemos abierto tiene una sola variable: `air`. Para extraerla, podemos usar algunas notaciones

In [ ]:
ds.air

In [ ]:
ds["air"]

Si bien ambas notaciones son equivalente, la notacion "punto", i.e. `ds.air`, no funcionara si la variable tiene el nombre de alguna de las funciones incorporadas en python

In [ ]:
air = ds["air"]

Operaciones básicas de agregación disponibles en numpy estan disponibles en `xarray`

In [ ]:
air.mean()

In [ ]:
air.std()

In [ ]:
air.median()

Sin embargo, esto no impide que usemos operaciones de `numpy` sobre estructuras de `xarray`

In [ ]:
np.mean(air)

## 3. Trabajando con datos

### 3.1. Broadcasting y Alignment

Uno de los conceptos importantes al trabajar con arreglos de `numpy` es la propagacion de dimensiones o *broadcasting*.


<div align="center">
<img src="http://scipy-lectures.org/_images/numpy_broadcasting.png" width=50%>
</div>

<a style="font-size: 0.8em" href="http://scipy-lectures.org/intro/numpy/operations.html#broadcasting">Scipy Lecture Notes</a>

In [ ]:
a = xr.DataArray(np.arange(3), dims="time", coords={"time": np.arange(3)})
a

In [ ]:
b = xr.DataArray(np.arange(4), dims="space", coords={"space": np.arange(4)})
b

In [ ]:
a + b

Por otro lado, la lineacion de coordenadas o *alignment* es una operacion que xarray realiza cuando encuentra coordenadas en comun entre dos o más `DataArray`

<div align="center">
<img src="https://xarray-contrib.github.io/xarray-tutorial/_images/broadcasting.png" width=50%>
</div>

<a style="font-size: 0.8em" href="https://docs.google.com/presentation/d/16CMY3g_OYr6fQplUZIDqVtG-SKZqsG8Ckwoj2oOqepU/edit#slide=id.g67406e8d7_0_135">Imagen de Stephan Hoyer</a>

In [ ]:
a = air.sel(time=slice("2013-01-01", "2013-12-31")).isel(lat=0)
a

In [ ]:
b = air.sel(time=slice("2013-10-01", "2014-12-31")).isel(lat=10)
b

In [ ]:
a + b

Este tipo de operaciones permite mantener los datos en la coordenada que les corresponde, mantiendo todo siempre en la misma grilla.

### 3.2. Operaciones de alto nivel

Tomado de [xarray in 45 minutes](https://xarray-contrib.github.io/xarray-tutorial/oceanhackweek-2020/xarray-oceanhackweek20.html#High-level-computation:-groupby,-resample,-rolling,-coarsen,-weighted)

`xarray` cuenta con algunos objetos de alto nivel que permiten realizar operaciones comunes

- groupby : Agrupa y reduce datos
- resample : Groupby especializado para ejes de tiempo. Puede aumentar o reducir la resolución temporal.
- rolling : Aplica ventanas moviles sobre los datos e.g. media corrida
- coarsen : Reduce la resolución de los datos
- weighted : Aplica pesos previo a un operación de reducción

`groupby` es altamente útil en la dimensión del tiempo, especialmente para calcular climatologías

In [ ]:
air.groupby("time.day")

In [ ]:
air.groupby("time.month")

In [ ]:
air.groupby("time.month").mean()

`resample` nos permite cambiar la frecuencia en la dimensión del tiempo

In [ ]:
air.resample(time="1D")

In [ ]:
air.resample(time="1D").mean()

`rolling` permite construir operaciones sobre ventanas móviles

In [ ]:
air_ts = air.isel(lat=10, lon=10)
air_ts.rolling(time=30)

In [ ]:
air_ts.plot(label="orig")
air_ts.rolling(time=30).mean().plot(label="rolling")
plt.legend()

### 3.3. Otras operaciones

`xarray` provee muchas operaciones de conveniencia. Siempre es una buena idea revisar la documentación para ver todo lo que tiene por ofrecer. [Link](http://xarray.pydata.org/en/stable/api.html)

In [ ]:
air.differentiate("time")

In [ ]:
air.integrate("time")

In [ ]:
air.isel(lat=10, lon=10).polyfit("time", deg=2)

## 4. Gráficos

Para una descripción más detallada, revisar https://xarray.pydata.org/en/stable/plotting.html

`xarray` permite realizar plots simples de nuestros datos usando el método `.plot`

In [ ]:
air.plot()

Es posible hacer una grilla de gráficos especificando algunos parámetros

In [ ]:
air.isel(time=slice(100, 105)).plot(col="time")

In [ ]:
air.isel(time=slice(100, 105)).plot.contourf(col="time")

## 5. Integración con dask

`xarray` tiene la capacidad de leer multiples archivos netcdf dentro de un solo objeto mediante la función `open_mfdataset`. Ya que no contamos con una gran cantidad de datos por el momento, vamos a simular este procedimiento aplicando `chunks` a nuestros datos de temperatura

In [ ]:
air_dask = air.chunk(dict(time=4))
air_dask

Las operaciones a realizar sobre esta nueva variable seran evaluadas de manera "vaga" (*lazy*), esto quiere decir que ninguna operación será realizada hasta que se llame `compute` o `load`

In [ ]:
air_dask.mean(dim="lat")

In [ ]:
air_dask.groupby("time.month").mean(dim="time")

## 6. Aplicación con datos reales

Para cargar datos en formato netcdf a una variable, se hacer uso de la función open_dataset (en el caso de tener 1 solo archivo) o open_mfdataset en caso de tener muchos archivos. Es este caso vamos a leer datos de temperatura superficial del mar de ERSSTv5 para hacer el cálculo del Oceanic Niño Index (ONI).

In [ ]:
ersstv5 = xr.open_dataset(
    "https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"
)
ersstv5

### Calculo del ONI

El ONI esta definido como la media corrida de 3 meses sobre el indice del Niño 3.4 (5°S-5°N / 170°W-120°W)

In [ ]:
nino34 = ersstv5.sst.sel(lat=slice(5, -5), lon=slice(190, 240))
nino34

In [ ]:
nino34 = nino34.mean(dim=["lat", "lon"])

#### Cálculo de la climatología

Para realizar este cálculo, vamos a tomar como periodo base 1986-2015 con fines ilustrativos. En el cálculo actual, el periodo de la climatología cambia cada 5 años segun los criterios considerados por el centro de predicción del clima de la NOAA ([ref](https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_change.shtml))

In [ ]:
nino34_clim = (
    nino34.sel(time=slice("1986-01-01", "2015-12-31"))
    .groupby("time.month")
    .mean(dim="time")
)
nino34_clim

In [ ]:
fig, ax = plt.subplots()
nino34_clim.plot(ax=ax)
ax.set_yticks(np.arange(26, 28.1, 0.2))
ax.set_ylim(26, 28)
ax.grid(ls="--")

#### Cálculo de la anomalía

Utilizaremos una sintaxis similar para restar las climatologías a los valores absolutos

In [ ]:
nino34_anom = nino34.groupby("time.month") - nino34_clim
nino34_anom

In [ ]:
ONI = nino34_anom.rolling(time=3, center=True).mean()

# truncamos los valores al segundo decimal
ONI = np.around(ONI, decimals=2)
ONI

In [ ]:
ONI.attrs["long_name"] = "SST Anomaly"
ONI.attrs["units"] = "degrees C"

In [ ]:
ONI.plot()

Con algo de tiempo, podemos hacer un gráfico un poco más atractivo

In [ ]:
# Colocamos el tiempo que queremos plotear
plot_data = ONI.sel(time=slice("2008-01-01", "2019-12-31"))

# Declaramos la figura y los ejes
fig, ax = plt.subplots(dpi=200)

ax.fill_between(
    plot_data.time.data,
    plot_data.data,
    0.5,
    where=plot_data > 0.5,
    color="red",
    interpolate=True,
    lw=0,
    label="ONI > 0.5",
)
ax.fill_between(
    plot_data.time.data,
    plot_data.data,
    -0.5,
    where=plot_data < -0.5,
    color="blue",
    interpolate=True,
    lw=0,
    label="ONI < 0.5",
)
plot_data.plot(ax=ax, c="k", lw=0.5)

minx = plot_data.time.min().data
maxx = plot_data.time.max().data
ax.hlines(0.5, minx, maxx, lw=0.5, linestyles="--")
ax.hlines(-0.5, minx, maxx, lw=0.5, linestyles="--")

ax.set_ylim(-4, 4)
ax.set_xlim(minx, maxx)

ax.text(
    0.99,
    0.01,
    "Clim 1981-2010",
    fontsize=5,
    horizontalalignment="right",
    transform=ax.transAxes,
)

ax.set_title("ERSSTv5 ONI")
ax.grid(ls="--", lw=1, alpha=0.3)
ax.legend()

## 7. Aplicación Numérica

*Tomado de un ejercicio del curso "Modelado Numérico de la Atmósfera" organizado por el IGP [[Repo]](https://github.com/DangoMelon/cmna-igp)*

Dada la siguiente malla de datos de temperatura, pronosticar la evolución del campo de la variable para una hora. Aplicar el método de Euler para las derivadas zonal y meridional.

### Datos
* Velocidad del viento = CONST = 7m/s
* Dirección del viento = CONST = 140°
* Resolución del dominio dx = dy = 18km
* Paso del tiempo -> por determinar

$$
TEMP = \begin{bmatrix}
21.2 & 22.8 & 23.5 & 24.8 & 26.5 \\
22.7 & 24.4 & 25.2 & 26.5 & 28.4 \\
24.3 & 26.1 & 27.0 & 28.4 & 30.4 \\
26.0 & 27.9 & 28.9 & 30.4 & 32.5 \\
27.8 & 29.9 & 30.9 & 32.5 & 34.8 \\
29.7 & 32.0 & 33.0 & 34.8 & 37.2 \\
31.8 & 34.2 & 35.3 & 37.2 & 39.8
\end{bmatrix}$$

---

## Solución

La ecuación que estamos tratando de resolver es la siguiente:

$$
\frac{\partial T }{\partial t} + u \frac{\partial T}{\partial x} + v \frac{\partial T}{\partial y} = 0
$$

En donde el paso de tiempo estara determinado por el numero de Courant

$$
\frac{\lvert u\rvert*\Delta t}{\Delta x} + \frac{\lvert v\rvert*\Delta t}{\Delta y} \leq 0.06
\\
$$

Que resulta en

$$
\Delta t \leq \frac{0.06}{\left(\frac{\lvert u\rvert}{\Delta x} + \frac{\lvert v\rvert}{\Delta y}\right)}
$$

In [ ]:
# Graficos interactivos
import holoviews as hv
import panel as pn
from panel.interact import fixed, interact

pn.extension()
hv.extension("bokeh")

In [ ]:
# Declaración de datos
dx = dy = 18
magw = 7
angw = 140
temp_arr = np.array(
    [
        [21.2, 22.8, 23.5, 24.8, 26.5],
        [22.7, 24.4, 25.2, 26.5, 28.4],
        [24.3, 26.1, 27.0, 28.4, 30.4],
        [26.0, 27.9, 28.9, 30.4, 32.5],
        [27.8, 29.9, 30.9, 32.5, 34.8],
        [29.7, 32.0, 33.0, 34.8, 37.2],
        [31.8, 34.2, 35.3, 37.2, 39.8],
    ]
)

In [ ]:
# Calculo de las componentes u,v
u = magw * np.cos(np.deg2rad(140))
v = magw * np.sin(np.deg2rad(140))

print(f"u = {u}\n v = {v}")

Con los valores obtenidos para u y v, calculamos el valor límite que debe tener $\Delta t$

In [ ]:
dt_approx = 0.06 / ((np.abs(u) / (dx * 1000)) + (np.abs(v) / (dy * 1000)))
print(f"dt_approx = {dt_approx}")

Entonces tomamos

$$
\Delta t \approx 100s
$$

Construimos una grilla que cumpla con $\Delta x$ y $\Delta y$ (no es necesario realizar esto, pero es util para tener coordenadas definidas dentro de xarray)

In [ ]:
lon = np.arange(-90, -90 + temp_arr.shape[1] * dx, dx)
lat = np.arange(-50, -50 + temp_arr.shape[0] * dy, dy)
print(f"lon = {lon}\nlat = {lat}")

Construimos nuestra matriz de datos con `lat` y `lon` como coordenadas

In [ ]:
temp = xr.DataArray(temp_arr, coords=[("lat", lat[::-1]), ("lon", lon)])

# Metadata adicional para variables/coordenadas que sigue la convención CF
temp.lon.attrs["long_name"] = "Longitude"
temp.lon.attrs["units"] = "degrees_east"
temp.lat.attrs["long_name"] = "Latitude"
temp.lat.attrs["units"] = "degrees_north"
temp.attrs["long_name"] = "Temperature"
temp.attrs["units"] = "degrees_C"
temp.name = "temperature"

temp

In [ ]:
# Arreglos para grafico de quivers
u_arr = np.full_like(temp_arr, u)
v_arr = np.full_like(temp_arr, v)

Se puede explorar nuestra malla de datos iniciales

In [ ]:
fig, ax = plt.subplots(dpi=100)
temp.plot(ax=ax, vmin=20, vmax=40)
ax.quiver(lon, lat, u_arr, v_arr)
ax.set_title("En grilla")

In [ ]:
levels = np.arange(20, 40.5, 0.5)

fix, ax = plt.subplots(dpi=100)
temp.plot.contourf(ax=ax, vmin=20, vmax=40, levels=levels)
CS = temp.plot.contour(ax=ax, vmin=20, vmax=40, colors="k", levels=levels[::4])
ax.quiver(lon + dx / 2, lat + dy / 2, u_arr, v_arr)
ax.set_ylim(lat.min(), lat.max())
ax.set_xlim(lon.min(), lon.max())
ax.clabel(CS, colors="k", fmt="%i")
ax.set_title("En contornos")

---

La discretización de la ecuacion a resolver se hara con un esquema centrado en el espacio y adelantado en el tiempo

$$
\left(\frac{T_{i,j}^{n+1} - T_{i,j}^{n}}{\Delta t}\right) + u \left(\frac{T_{i+1,j}^{n} - T_{i-1,j}^{n}}{2\Delta x} \right) + v \left( \frac{T_{i,j+1}^{n} - T_{i,j-1}^{n}}{2\Delta y} \right) = 0
$$

De donde

$$
T_{i,j}^{n+1} = \Delta t \ T_{i,j}^{n} - \frac{u \Delta t}{2 \Delta x} \left( T_{i+1,j}^{n} - T_{i-1,j}^{n} \right) - \frac{v \Delta t}{2 \Delta y} \left( T_{i,j+1}^{n} - T_{i,j-1}^{n} \right)
$$

Implementamos la discretizacion obtenida en una función que nos permitirá obtener la temperatura pronosticada

In [ ]:
def solve(
    inarr,
    dt,
    u,
    v,
    nt,
    boundary="1st_order",
    tstart="2021-04-03T00:00",
    plot=False,
    dynamic=False,
    **hv_kwargs,
):
    """
    Resuelve la ecuacion de momento en dos dimensiones con velocidades constantes

    Parametros
    ----------
    inarr: DataArray
        Objeto de xarray con grilla regular (lat, lon) que
        representa el campo inicial
    dt: int
        Salto de tiempo
    u: float
        Valor de la velocidad zonal
    v: float
        Valor de la velocidad meridional
    nt: int
        Saltos de tiempo a resolver
    boundary: {'1st_order', '2nd_order', 'constant'}
        Condicion de frontera a usar.
        Por defecto se usa '1st_order'
        Puede ser una de las siguientes opciones:
            -'1st_order': aproximacion de primer orden hacia
                adelante/atras en los bordes
            -'2nd_order': aproximacion de segundo orden hacia
                adelante/atras en los bordes
            -'constant': frontera constante, i.e dT/dr = 0
    tstart: string, opcional
        Fecha inicial de la simulación. Útil para la dimension
        tiempo a agregar en el array de salida.
        Por defecto toma el valor 2021-04-03T00:00

    Retorna
    -------
    outarr: DataArray
        Arreglo con las mismas coordenadas que inarr pero con
        la dimension `time` agregada, de longitud `nt`+1 y
        saltos `dt`.
    """

    boundaries = {"1st_order": 1, "2nd_order": 2}
    diff_kwargs = {}

    tdelta = pd.Timedelta(dt, unit="s")
    tstart = pd.to_datetime(tstart)
    outarr = xr.DataArray(
        np.nan,
        coords=[
            ("time", pd.date_range(tstart, tstart + (tdelta * nt), freq=f"{dt}s")),
            ("lat", inarr.lat),
            ("lon", inarr.lon),
        ],
    )
    outarr[0] = inarr

    const = 1
    if boundary in boundaries.keys():
        diff_kwargs = dict(edge_order=boundaries[boundary])
    elif boundary == "constant":
        const = xr.zeros_like(inarr)
        const[1:-1, 1:-1] = 1

    for i in range(outarr.time.size - 1):
        current_time = outarr.isel(time=i)
        cdiffx = current_time.differentiate("lon", **diff_kwargs) * const / 1000
        cdiffy = current_time.differentiate("lat", **diff_kwargs) * const / 1000
        outarr[i + 1] = (current_time) - (u * dt * cdiffx) - (v * dt * cdiffy)

    # Colocamos la metadata
    outarr["boundary"] = boundary
    outarr = outarr.expand_dims("boundary")
    outarr.lat.attrs = inarr.lat.attrs
    outarr.lon.attrs = inarr.lon.attrs
    outarr.time.attrs["long_name"] = "Time"
    outarr.boundary.attrs["long_name"] = "Boundary Condition"
    outarr.attrs = inarr.attrs
    outarr.name = "forecast_temp"

    # En caso de querer retornar el gráfico
    if plot:
        hmap = hv.Dataset(outarr).to(hv.Image, kdims=["lon", "lat"])
        cont = hv.operation.contours(
            hmap, filled=True, dynamic=dynamic, levels=np.arange(-5, 60, 1.0).tolist()
        ).redim.range(forecast_temp=(20, 40))
        xx, yy = np.meshgrid(
            np.linspace(lon.min(), lon.max(), 30), np.linspace(lat.min(), lat.max(), 30)
        )
        vector = hv.VectorField(
            (
                xx,
                yy,
                np.full_like(xx, np.arctan2(v, u)),
                np.full_like(xx, 1),
            )
        )
        layout = cont * vector
        layout.opts(
            hv.opts.Polygons(
                tools=["hover", "save"],
                default_tools=[],
                ylim=(lat.min(), lat.max()),
                xlim=(lon.min(), lon.max()),
                frame_width=500,
                aspect="equal",
                colorbar=True,
                color_levels=22,
            ),
            hv.opts.Overlay(default_tools=[]),
            hv.opts.VectorField(default_tools=[]),
        )
        if dynamic:
            pan = pn.panel(layout, widget_type="scrubber", widget_location="bottom")
            # Modificamos la velocidad de reproducción por defecto
            player = pan[1][1][0]
            player.param.interval.default = 150
            return pan
        return layout

    return outarr

In [ ]:
fore_temp = solve(temp, 100, u, v, 36, boundary="2nd_order")
fore_temp

Exploramos el resultado para condiciones de frontera con solución de segundo orden y constante

In [ ]:
fcst_2nd = solve(temp, 100, u, v, 36, plot=True, boundary="2nd_order")
fcst_2nd

In [ ]:
fcst_const = solve(temp, 100, u, v, 36, plot=True, boundary="constant")
fcst_const

---

## Live Demo

In [ ]:
live_panel = interact(
    solve,
    inarr=fixed(temp),
    dt=(10, 110, 10, 100),
    u=(-15, 15, 0.1, u),
    v=(-15, 15, 0.1, v),
    nt=(18, 36 * 5, 18, 36),
    plot=fixed(True),
    boundary=["1st_order", "2nd_order", "constant"],
    dynamic=fixed(True),
)

pn.Row(
    pn.layout.HSpacer(),
    pn.Column(
        pn.pane.PNG(
            "https://repositorio.igp.gob.pe/themes/Mirage/images/logo_igp_sharecontent.png",
            align="center",
            width=250,
            margin=-25,
        ),
        pn.pane.Markdown("## Difusión de la Temperatura\n **Parámetros**", margin=0),
        live_panel[0],
        pn.layout.VSpacer(),
        pn.pane.Markdown("*Club de Python para Físicos*"),
        margin=20,
    ),
    live_panel[1],
    pn.layout.HSpacer(),
)


## 8. Recursos útiles

* [xarray Tutorial](https://xarray-contrib.github.io/xarray-tutorial/index.html)
* [Pangeo](https://pangeo.io/)
* [Scipy Lecture Notes](http://scipy-lectures.org/index.html)
* ["Scaling Scientific Python"](https://matthewrocklin.com/slides/ams-2018.html#/) por Matthew Rocklin
* [Documentación](http://xarray.pydata.org/en/stable/index.html) oficial de xarray
* ["multi-dimensional data analysis in Python"](https://fabienmaussion.info/acinn_xarray_workshop/#/) por Fabien Maussion
* [An Introduction to Earth and Environmental Data Science](https://earth-env-data-science.github.io/intro)
## Donaciones
<br>
<center>
Puedes donar una vez en el siguiente enlace (Ko-Fi):

<br>
    
*Click en la imagen.*

<a href="https://ko-fi.com/rcrdphysics">
<img src="https://habrastorage.org/webt/8r/ml/xf/8rmlxfpdzukegpxa62cxlfvgkqe.png" width=20% />

</center>